In [1]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
df_1 = pd.read_csv("C:/00ETH/ml4h/Project1/archive/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("C:/00ETH/ml4h/Project1/archive/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [3]:
def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [5]:
model = get_model()
file_path = "baseline_cnn_ptbdb.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 183, 16)           96        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 89, 16)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 89, 16)            0         
__________________________________________________________

In [6]:
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000
 - 5s - loss: 0.5205 - acc: 0.7441 - val_loss: 0.5412 - val_acc: 0.7665

Epoch 00001: val_acc improved from -inf to 0.76652, saving model to baseline_cnn_ptbdb.h5
Epoch 2/1000
 - 4s - loss: 0.3488 - acc: 0.8535 - val_loss: 0.2909 - val_acc: 0.8798

Epoch 00002: val_acc improved from 0.76652 to 0.87983, saving model to baseline_cnn_ptbdb.h5
Epoch 3/1000
 - 4s - loss: 0.2824 - acc: 0.8881 - val_loss: 0.2235 - val_acc: 0.9167

Epoch 00003: val_acc improved from 0.87983 to 0.91674, saving model to baseline_cnn_ptbdb.h5
Epoch 4/1000
 - 4s - loss: 0.2416 - acc: 0.9089 - val_loss: 0.2200 - val_acc: 0.9116

Epoch 00004: val_acc did not improve from 0.91674
Epoch 5/1000
 - 4s - loss: 0.2189 - acc: 0.9173 - val_loss: 0.2171 - val_acc: 0.9159

Epoch 00005: val_acc did not improve from 0.91674
Epoch 6/1000
 - 5s - loss: 0.1932 - acc: 0.9246 - val_loss: 0.1625 - val_acc: 0.9373

Epoch 00006: val_acc improved from 0.91674 to 0.93734, savi